Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

## Problem Description

random notes: 

- GOAL: maximizing the value and minimizing the weight of objects carried by the <NUM_KNAPSACKS> knapsacks

- We should respect the contraints on the weight, which defines how much weights a knapsack can carry.

- a dimension defines a contraint on a napsack.
Ex. [x, y, z] -> 3 constraints on backpack 1.

- The solution is represented by a 2D matrix (NUM_KNAPSACKS, NUM_ITEMS). Each row represents a knapsack and for each row we have a list of carried items where its length = NUM_ITEMS. Each element of the list is either a 1 if the knapsack carries the item, or is 0 if it doesn't carry the item.

- the fitness of the function is defined by overall value carried by all the knapsacks combined (sum of single total value of each knapsack)

In [1]:
import numpy as np
from random import randint
import math

In [2]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [3]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [4]:
CONSTRAINTS

array([[240, 138],
       [262, 262],
       [105, 118]])

In [5]:
# A random solution - starting point
solution = np.array(
    [np.random.random(NUM_ITEMS) < 0.5 for _ in range(NUM_KNAPSACKS)], dtype=np.bool
)

### UTILS

In [6]:
def get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS):
    return {
        "NUM_KNAPSACKS": NUM_KNAPSACKS,
        "NUM_ITEMS": NUM_ITEMS,
        "NUM_DIMENSIONS": NUM_DIMENSIONS,
        "VALUES": VALUES,
        "WEIGHTS": WEIGHTS,
        "CONSTRAINTS": CONSTRAINTS,
    }

In [7]:
def generate_random_solution(num_knapsacks, num_items):
    """
    generate a pure random solution. In this case is possible to start
    from a starting point where you have items that are carried by more than 1 knapsack.
    """
    
    solution = np.array(
        [np.random.random(num_items) < 0.5 for _ in range(num_knapsacks)], dtype=np.int8
    )

    return solution

In [8]:
def generate_clean_random_solution(num_knapsacks, num_items):
    """
    generate a random solution that satisfy the first constraint
    (that is each item is either not carried or carried from exactly 1 item).
    """

    # generate a 2D matrix (num_knapsacks, num_items) full of 0s.
    solution = np.zeros((num_knapsacks, num_items), dtype=int)

    # iterate over the columns
    for item_idx in range(num_items):

        # generate random number:
        # if 0 the item is not carried
        # if 1 the item is carried by 1 knapsack
        carried = randint(0, 1)
    
        if carried == 0:
            continue
        
        # generate a random index that identifies the knapsack that carries the item
        id_knapsack = randint(0, num_knapsacks - 1)

        # update the solution
        solution[id_knapsack, item_idx] = 1
        
    return solution

In [9]:
def fitness(solution, problem):
    """
    Calculates the fitness of a solution. It is defined by the overall valued
    carried by all knapsacks.
    """

    # check the basic constraint: an item cannot be carried by more then 1 knapsack
    # if the sum of the items in the same column >= 1 it means that
    # at leats 2 knapsacks are carrying the item, so a fitness of 0 is returned
    if np.any(np.sum(solution, axis=0) > 1):
        return 0

    values = problem["VALUES"]
    weights = problem["WEIGHTS"]
    constraints = problem["CONSTRAINTS"]

    # sum all the values for each row based on the items that each knapsack carries
    total_value = np.sum(solution * values)
    # get the total weights for each knapsack
    knapsack_loads = solution @ weights

    # Check if any constraint is violated
    if np.any(knapsack_loads > constraints):
        return 0 
    else:
        return total_value

In [10]:
def tweak(solution, problem):
    """
    Creates a "neighbor" solution by making one small, random change.
    It randomly picks one item and changes its knapsack assignment.
    """
    # get problem parameters
    num_items = problem["NUM_ITEMS"]
    num_knapsacks = problem["NUM_KNAPSACKS"]
    
    # create a copy to avoid modifying the original solution
    neighbor_solution = solution.copy()
    
    # pick a random item to move
    item_to_move = np.random.randint(num_items)
    # pick a random new knapsack for it (-1 means removing it)
    new_knapsack = np.random.randint(-1, num_knapsacks)
    
    # first, remove the item from its old position (if any)
    neighbor_solution[:, item_to_move] = 0
    # if the item is being assigned to a new knapsack (not just removed)
    if new_knapsack != -1:
        # place the item in the new knapsack
        neighbor_solution[new_knapsack, item_to_move] = 1
        
    return neighbor_solution

### ALGORITHMS

In [11]:
def hill_climbing(problem, max_iterations=10000):
    """
    Performs Hill Climbing.
    """

    # get the variables from the given problem
    num_knapsacks = problem["NUM_KNAPSACKS"]
    num_items = problem["NUM_ITEMS"]

    # generate a random solution (starting point)
    current_solution = generate_clean_random_solution(num_knapsacks, num_items)

    # calculate the fitness for the current_solution
    current_fitness = fitness(current_solution, problem)
    print(f"Starting Hill Climbing with initial fitness: {current_fitness}")

    for i in range(max_iterations):
        # *** REFACTORED PART ***
        # create a neighbor solution by calling the new tweak function
        neighbor_solution = tweak(current_solution, problem)

        # evaluate the neighbor by calculating its fitness
        neighbor_fitness = fitness(neighbor_solution, problem)

        # if the neighbor is better, move to it
        if neighbor_fitness > current_fitness:
            current_solution = neighbor_solution
            current_fitness = neighbor_fitness
            print(f"Iteration {i}: Found better solution with fitness {current_fitness}")

    print("\nHill Climbing finished")
    return current_solution, current_fitness

In [12]:
def simulated_annealing(problem, initial_temp, final_temp, cooling_rate):
    """
    Performs a Simulated Annealing search to solve the MMKP.
    """
    num_knapsacks = problem["NUM_KNAPSACKS"]
    num_items = problem["NUM_ITEMS"]

    # Start with a valid random solution
    current_solution = generate_clean_random_solution(num_knapsacks, num_items)
    current_fitness = fitness(current_solution, problem)

    # We need to keep track of the best solution found so far
    best_solution = current_solution
    best_fitness = current_fitness

    temp = initial_temp
    print(f"Starting SA with initial fitness: {current_fitness}, Temp: {temp:.2f}")

    iteration = 0
    while temp > final_temp:
        # *** REFACTORED PART ***
        # Create a neighbor solution by calling the new tweak function
        neighbor_solution = tweak(current_solution, problem)

        neighbor_fitness = fitness(neighbor_solution, problem)
        
        # Calculate the change in fitness
        delta_fitness = neighbor_fitness - current_fitness

        # Core of the SA algorithm: The Acceptance Criterion
        if delta_fitness > 0:
            # If the neighbor is better, always accept it
            current_solution = neighbor_solution
            current_fitness = neighbor_fitness
        else:
            # If the neighbor is worse, accept it with a certain probability
            # The probability decreases as temperature drops
            acceptance_probability = math.exp(delta_fitness / temp)
            if np.random.random() < acceptance_probability:
                current_solution = neighbor_solution
                current_fitness = neighbor_fitness

        # Update the overall best solution found
        if current_fitness > best_fitness:
            best_solution = current_solution
            best_fitness = current_fitness
            print(f"Iter {iteration}: New best fitness -> {best_fitness} (Temp: {temp:.2f})")
            
        # Cool the temperature
        temp *= cooling_rate
        iteration += 1

    print("\nSimulated Annealing finished.")
    return best_solution, best_fitness

## TEST PROBLEMS

In [13]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# getting problem p1
p1 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

In [14]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

p2 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

In [15]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

p3 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

#### TEST - HILL CLIMBING

In [16]:
print("TEST - HILL CLIMBING")
print("PROBLEM 1")
best_solution, best_fitness = hill_climbing(p1, max_iterations = 20000)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

TEST - HILL CLIMBING
PROBLEM 1
Starting Hill Climbing with initial fitness: 0
Iteration 20: Found better solution with fitness 528
Iteration 23: Found better solution with fitness 611
Iteration 26: Found better solution with fitness 688
Iteration 27: Found better solution with fitness 764
Iteration 31: Found better solution with fitness 829
Iteration 32: Found better solution with fitness 914
Iteration 42: Found better solution with fitness 923
Iteration 51: Found better solution with fitness 943
Iteration 69: Found better solution with fitness 994
Iteration 102: Found better solution with fitness 1065

Hill Climbing finished
Best fitness: 1065
Best solution:
 [[0 0 0 0 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 0]
 [0 1 1 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1]]


In [17]:
print("PROBLEM 2")
best_solution, best_fitness = hill_climbing(p2, max_iterations = 20000)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

PROBLEM 2
Starting Hill Climbing with initial fitness: 0

Hill Climbing finished
Best fitness: 0
Best solution:
 [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0

In [ ]:
print("PROBLEM 3")
best_solution, best_fitness = hill_climbing(p3, max_iterations = 100)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

### TEST - SIMULATED ANNILING

In [19]:
print("TEST - SIMULATED ANNEALING")

INITIAL_TEMPERATURE = 100.0
FINAL_TEMPERATURE = 0.1
COOLING_RATE = 0.999

TEST - SIMULATED ANNEALING


In [20]:
print("PROBLEM 1")
best_solution, best_fitness = simulated_annealing(p1, INITIAL_TEMPERATURE, FINAL_TEMPERATURE, COOLING_RATE)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

PROBLEM 1
Starting SA with initial fitness: 649, Temp: 100.00
Iter 3: New best fitness -> 698 (Temp: 99.70)
Iter 5: New best fitness -> 783 (Temp: 99.50)
Iter 6: New best fitness -> 860 (Temp: 99.40)
Iter 9: New best fitness -> 880 (Temp: 99.10)
Iter 28: New best fitness -> 883 (Temp: 97.24)
Iter 30: New best fitness -> 892 (Temp: 97.04)
Iter 33: New best fitness -> 944 (Temp: 96.75)
Iter 94: New best fitness -> 1016 (Temp: 91.02)
Iter 124: New best fitness -> 1037 (Temp: 88.33)
Iter 1734: New best fitness -> 1048 (Temp: 17.64)
Iter 2036: New best fitness -> 1057 (Temp: 13.04)
Iter 2061: New best fitness -> 1065 (Temp: 12.72)

Simulated Annealing finished.
Best fitness: 1065
Best solution:
 [[1 0 0 0 1 1 1 0 1 1 0 1 0 0 1 0 0 1 0 1]
 [0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 1 0]
 [0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0]]


In [21]:
print("PROBLEM 2")
best_solution, best_fitness = simulated_annealing(p2, INITIAL_TEMPERATURE, FINAL_TEMPERATURE, COOLING_RATE)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

PROBLEM 2
Starting SA with initial fitness: 0, Temp: 100.00

Simulated Annealing finished.
Best fitness: 0
Best solution:
 [[0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0

In [ ]:
print("PROBLEM 3")
best_solution, best_fitness = simulated_annealing(p3, INITIAL_TEMPERATURE, FINAL_TEMPERATURE, COOLING_RATE)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

### CONSIDERATIONS

So far we have tried both the hill climbing and the simulated annealing on all the 3 problems.

RESULTS:
Both the algorithms achieve a feasible result for problem 1, but they fail when it comes to finding a solution for problems 2 and 3.
This is due to the small set of KNAPSACKS, ITEMS and DIMENSIONS that permits to find a better solution after some iterations even if we start from an invalid solution.
With Problems 2 and 3 we likely start with an invalid solution in an area that is far away from a feasible solution, for this reason even after a lot of iteration we don't have an improvement.

The next algorithm tried is the iterated local search

In [23]:
def local_search_hc(solution, problem, max_steps=500):
    """
    Hill climbing algorithm, which is a sub component
    of the iterated local search
    """
    current_solution = solution.copy()
    current_fitness = fitness(current_solution, problem)

    for _ in range(max_steps):
        # *** REFACTORED PART ***
        # create a neighbor solution by calling the new tweak function
        neighbor_solution = tweak(current_solution, problem)

        neighbor_fitness = fitness(neighbor_solution, problem)

        if neighbor_fitness > current_fitness:
            current_solution = neighbor_solution
            current_fitness = neighbor_fitness
    
    return current_solution, current_fitness


def perturb_solution(solution, problem, strength=0.1):
    """
    the current solution is forced to change randomically
    """
    perturbed_solution = solution.copy()
    num_items = problem["NUM_ITEMS"]
    
    # it defines how many time we want to perturb our solution
    num_to_perturb = int(num_items * strength)

    # at each iteration we apply one tweak to the solution
    for _ in range(num_to_perturb):
        # *** REFACTORED PART ***
        # apply one random change using the tweak function
        perturbed_solution = tweak(perturbed_solution, problem)
    
    return perturbed_solution



def iterated_local_search(problem, max_iterations=100, perturbation_strength=0.2):
    """
    Performs an Iterated Local Search.
    (This function does not need changes, as it calls the refactored functions above)
    """
    # get the variables from the given problem
    num_knapsacks = problem["NUM_KNAPSACKS"]
    num_items = problem["NUM_ITEMS"]

    # start with a good initial solution
    initial_solution = generate_clean_random_solution(num_knapsacks, num_items)
    
    # find the first local optimum
    best_solution, best_fitness = local_search_hc(initial_solution, problem)
    print(f"ILS Initial Best Fitness: {best_fitness}")

    for i in range(max_iterations):
        # perturb the best solution found so far
        changed_solution = perturb_solution(best_solution, problem, strength=perturbation_strength)
        
        # run local search on the new starting point
        new_solution, new_fitness = local_search_hc(changed_solution, problem)
        
        # acceptance criterion: if the new optimum is better, keep it
        if new_fitness > best_fitness:
            best_solution = new_solution
            best_fitness = new_fitness
            print(f"Iteration {i}: Found new best solution -> Fitness: {best_fitness}")
    
    print("\nIterated Local Search finished.")
    return best_solution, best_fitness

In [ ]:
print("PROBLEM 2")
best_solution, best_fitness = iterated_local_search(p2, max_iterations=10000)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

### CONSIDERATIONS 2

still not working.

As the last try, a new version of hill climbing has been implemented
This version of hill climbing solves 1 constraint at the time. It is organized in 2 phases.
- PHASE 1: It first make sure that the solution satisfies a specific constraint, then we keep iterating the process checking the solution for all the other constraint.
- PHASE 2: The hill climbing algorithm is applied an iteratively looks for a slightly better solution by making a tweak of only 1 element.


In [25]:
def guided_hill_climbing(problem, max_steps=10000):
    """
    This version of hill climbing focuses on 1 constraint at the time.
    It first repairs the solution to make it valid, then optimizes it for value.
    """
    num_knapsacks = problem["NUM_KNAPSACKS"]
    num_items = problem["NUM_ITEMS"]
    weights = problem["WEIGHTS"]
    values = problem["VALUES"]
    constraints = problem["CONSTRAINTS"]
    
    # start with a random solution
    current_solution = generate_clean_random_solution(num_knapsacks, num_items)
    
    # PHASE 1
    # taking an invalid solution and repair it
    print("Starting Repair Phase...")
    
    # loop until the solution becomes valid
    while True:
        current_loads = current_solution @ weights
        overload = current_loads - constraints
        overweight_knapsacks = np.where(np.any(overload > 0, axis=1))[0]
        
        if len(overweight_knapsacks) == 0:
            print("Solution is now valid. Moving to Optimization Phase.")
            break 
            
        knapsack_to_fix = overweight_knapsacks[0]
        items_in_knapsack = np.where(current_solution[knapsack_to_fix] == 1)[0]
        
        if len(items_in_knapsack) == 0:
            continue

        item_to_remove = np.random.choice(items_in_knapsack)
        current_solution[knapsack_to_fix, item_to_remove] = 0
        
    # PHASE 2
    # now the solution is valid, so we optimize it
    current_fitness = np.sum(current_solution * values)
    print(f"Initial valid fitness: {current_fitness}")
    
    # start the optimization loop for a fixed number of steps
    for i in range(max_steps):
        # *** REFACTORED PART ***
        # create a neighbor solution by calling the new tweak function
        neighbor_solution = tweak(current_solution, problem)
        
        # calculate the loads of this new neighbor solution
        neighbor_loads = neighbor_solution @ weights
        # check if the new solution violates any constraints
        if not np.any(neighbor_loads > constraints):
            # if it's valid, calculate its fitness
            neighbor_fitness = np.sum(neighbor_solution * values)

            # check if the neighbor's fitness is better than our current best
            if neighbor_fitness > current_fitness:
                current_solution = neighbor_solution
                current_fitness = neighbor_fitness
                print(f"Step {i}: Found better solution with fitness {current_fitness}")

    final_fitness = fitness(current_solution, problem)
    return current_solution, final_fitness

### TEST - HILL CLIMBING (check 1 constraint at the time)

In [26]:
print("PROBLEM 1")
best_solution, best_fitness = guided_hill_climbing(p1)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

PROBLEM 1
Starting Repair Phase...
Solution is now valid. Moving to Optimization Phase.
Initial valid fitness: 469
Step 5: Found better solution with fitness 512
Step 22: Found better solution with fitness 555
Step 25: Found better solution with fitness 567
Step 27: Found better solution with fitness 645
Step 30: Found better solution with fitness 714
Step 42: Found better solution with fitness 779
Step 48: Found better solution with fitness 855
Step 62: Found better solution with fitness 932
Step 219: Found better solution with fitness 941
Best fitness: 941
Best solution:
 [[0 0 1 1 1 0 0 1 0 1 0 1 0 1 1 1 0 0 0 0]
 [1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0]]


In [27]:
print("PROBLEM 2")
best_solution, best_fitness = guided_hill_climbing(p2)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

PROBLEM 2
Starting Repair Phase...
Solution is now valid. Moving to Optimization Phase.
Initial valid fitness: 18382
Step 1: Found better solution with fitness 19036
Step 2: Found better solution with fitness 19498
Step 10: Found better solution with fitness 20271
Step 14: Found better solution with fitness 20583
Step 18: Found better solution with fitness 21212
Step 23: Found better solution with fitness 21955
Step 24: Found better solution with fitness 22181
Step 26: Found better solution with fitness 22878
Step 29: Found better solution with fitness 23561
Step 32: Found better solution with fitness 24531
Step 39: Found better solution with fitness 25506
Step 65: Found better solution with fitness 26267
Step 73: Found better solution with fitness 26953
Step 76: Found better solution with fitness 27498
Step 80: Found better solution with fitness 28337
Step 116: Found better solution with fitness 28538
Step 126: Found better solution with fitness 28908
Step 167: Found better solution w

In [28]:
print("PROBLEM 3")
best_solution, best_fitness = guided_hill_climbing(p3, max_steps=1000)

print(f"Best fitness: {best_fitness}")
# print(f"Best solution:\n {best_solution}")

PROBLEM 3
Starting Repair Phase...


Solution is now valid. Moving to Optimization Phase.
Initial valid fitness: 979590
Step 4: Found better solution with fitness 980383
Step 6: Found better solution with fitness 980766
Step 7: Found better solution with fitness 981327
Step 12: Found better solution with fitness 982114
Step 13: Found better solution with fitness 982283
Step 17: Found better solution with fitness 982367
Step 20: Found better solution with fitness 983287
Step 26: Found better solution with fitness 983614
Step 34: Found better solution with fitness 984136
Step 47: Found better solution with fitness 984970
Step 48: Found better solution with fitness 985318
Step 54: Found better solution with fitness 985790
Step 61: Found better solution with fitness 985897
Step 63: Found better solution with fitness 986201
Step 77: Found better solution with fitness 986615
Step 81: Found better solution with fitness 986806
Step 87: Found better solution with fitness 987420
Step 90: Found better solution with fitness 988179
St